<a href="https://colab.research.google.com/github/NatLibFi/Annif/blob/kvp2019-workshop/arviopohjien_luonti_juho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install gspread-pandas

In [0]:
! mkdir -p  /root/.config/gspread_pandas/
! echo '{"installed":{"client_id":"257572009167-uai362klp88kr3hluabtoujf2hnd7fr6.apps.googleusercontent.com","project_id":"formal-archive-254810","auth_uri":"https://accounts.google.com/o/oauth2/auth","token_uri":"https://oauth2.googleapis.com/token","auth_provider_x509_cert_url":"https://www.googleapis.com/oauth2/v1/certs","client_secret":"x554t-fVXmFDrENKrXu8GuKy","redirect_uris":["urn:ietf:wg:oauth:2.0:oob","http://localhost"]}}' > /root/.config/gspread_pandas/google_secret.json

In [0]:
import pandas as pd
import random
from gspread_pandas import Spread, Client

In [0]:
client = Client()
spreads = client.find_spreadsheet_files_in_folders('Dokumenttien kuvailut')

df_kuvailut1 = pd.DataFrame()
df_kuvailut2 = pd.DataFrame()

#kuvailu_id = 0
#cols_to_keep = ['id', 'nimeke', 'url kokotekstiin', 'aiheet', 'kid', 'tiedostonimi']

for kuvailu_id, spread in enumerate(spreads['Dokumenttien kuvailut']):
  print(spread)
  # Tietokirjat:
  if spread['name'].startswith("kuvailut_tieto"):
      df_sheet = Spread(spread['id'], sheet=0).sheet_to_df(index=None)
      df_sheet['kid'] = kuvailu_id
      df_sheet['tiedostonimi'] = spread['name']
      df_sheet.rename(columns={'isbn': 'url kokotekstiin'}, inplace=True)
      df_kuvailut1 = df_kuvailut1.append(df_sheet) #[cols_to_keep])
  else:
      df_sheet = Spread(spread['id'], sheet=0).sheet_to_df(index=None)
      df_sheet['kid'] = kuvailu_id
      df_sheet['tiedostonimi'] = spread['name']
      df_sheet.columns = df_sheet.columns.str.lower()
      df_kuvailut2 = df_kuvailut2.append(df_sheet) #[cols_to_keep])    
  #kuvailu_id += 1
  #if kuvailu_id > 10: 
  #  break

{'id': '118wERQi6W_K6OCawjhxbDC2QtWmexL7uV6kDrtv7X9w', 'name': 'Copy of Kuvailut K16', 'path': '/'}
{'id': '118wERQi6W_K6OCawjhxbDC2QtWmexL7uV6kDrtv7X9w', 'name': 'Copy of Kuvailut K16', 'path': '/'}
Index(['id', 'nimeke', 'url kokotekstiin', 'aiheet', 'kid', 'tiedostonimi'], dtype='object')
{'id': '1agJ885Wj2Q1XgEiVgb_V2BVLbjVNhk0EyyFAmO1sEVI', 'name': 'kuvailut_tieto Osma', 'path': '/'}
Index(['id', 'nimeke', 'url kokotekstiin', 'aiheet', 'kid', 'tiedostonimi'], dtype='object')
{'id': '1C9K8zR3iMJyIxaq8dVRJmH5mWhuWqmhX32KCQZsmOIE', 'name': 'Copy of Kuvailut K31', 'path': '/'}
{'id': '1C9K8zR3iMJyIxaq8dVRJmH5mWhuWqmhX32KCQZsmOIE', 'name': 'Copy of Kuvailut K31', 'path': '/'}
Index(['id', 'nimeke', 'url kokotekstiin', 'aiheet', 'kid', 'tiedostonimi'], dtype='object')
{'id': '1VTUr0Jf6juzq60RzfoC6EytMBveHNbOAxAMXu7AJik8', 'name': 'kuvailut_tieto Mona', 'path': '/'}
Index(['id', 'nimeke', 'url kokotekstiin', 'aiheet', 'kid', 'tiedostonimi'], dtype='object')
{'id': '1rAcq6aVROxabfSJTwWk0-

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


{'id': '1eF1P_xfRFXjSzJ71zi1_aqEzlouyqfKmxCYEPKYXi5k', 'name': 'SKT4 VL Copy of Kuvailut ', 'path': '/'}
Index(['id', 'nimeke', 'url kokotekstiin', 'aiheet', 'yso-paikat aiheina',
       'yhteisöt aiheina', 'henkilöt aiheina', 'kid', 'tiedostonimi'],
      dtype='object')
{'id': '1V09_YiPfIPLQd0edJ1sr8i9Ap1gdXJx8ynYOF8SD8zU', 'name': 'SKT3 JM Kuvailut Copy of ', 'path': '/'}
{'id': '1V09_YiPfIPLQd0edJ1sr8i9Ap1gdXJx8ynYOF8SD8zU', 'name': 'SKT3 JM Kuvailut Copy of ', 'path': '/'}
Index(['id', 'nimeke', 'url kokotekstiin', 'aiheet', 'yso-paikat aiheina',
       'yhteisöt aiheina', 'henkilöt aiheina', 'kid', 'tiedostonimi'],
      dtype='object')
{'id': '19InLOfKZQE-O7_lFjKDCmf9xNnOGQLQVzgXGlnjjVIA', 'name': ' SKT1 KL Copy of Kuvailut', 'path': '/'}
{'id': '19InLOfKZQE-O7_lFjKDCmf9xNnOGQLQVzgXGlnjjVIA', 'name': ' SKT1 KL Copy of Kuvailut', 'path': '/'}
Index(['id', 'nimeke', 'url kokotekstiin', 'aiheet', 'yso-paikat aiheina',
       'yhteisöt aiheina', 'henkilöt aiheina', 'kid', 'tiedoston

In [0]:
print(df_sheet.columns)
print(df_kuvailut2.columns)

Index(['id', 'nimeke', 'url kokotekstiin', 'aiheet', 'kid', 'tiedostonimi'], dtype='object')
Index(['id', 'nimeke', 'url kokotekstiin', 'aiheet', '', '', '', '', 'kid',
       'tiedostonimi'],
      dtype='object')


In [168]:
df_kuvailut = pd.concat([df_kuvailut1, df_kuvailut2])
cols_to_keep = ['id', 'nimeke', 'url kokotekstiin', 'aiheet', 'kid', 'tiedostonimi']
df_kuvailut = df_kuvailut[cols_to_keep]
df_kuvailut.rename(columns={'id': 'did'}, inplace=True)
df_kuvailut[['did', 'nimeke', 'url kokotekstiin']] = df_kuvailut[['did', 'nimeke', 'url kokotekstiin']].mask(df_kuvailut[['did', 'nimeke', 'url kokotekstiin']]=='', None).ffill()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [0]:
ihminen = ['kuvailut_tieto Mona', 'jyx_kuvailut Mona', 'Copy of Kuvailut K']
annif_avusteiset = ['kuvailut_tieto Osma', 'kuvailut_tieto Juho', 'jyx_kuvailut_orig']
#annif = ['']
fennica = ['kuvailut_tieto Fennica', 'Copy of Kuvailut Fennica']

df_kuvailut['tyyppi'] = df_kuvailut['tiedostonimi'].apply(lambda x: 'annif_avusteinen' if x in annif_avusteiset
                                                                  else ('fennica' if x in fennica else 'ihminen'
                                                                  ))
df_kuvailut = df_kuvailut[df_kuvailut.did != 'D00']  # Poista esimerkkikuvailu

In [170]:
df_kuvailut

,did,nimeke,url kokotekstiin,aiheet,kid,tiedostonimi,tyyppi
0,TI01,"Mahtava moka : uskalla, opi ja menesty",9789521433085,virheet,1,kuvailut_tieto Osma,annif_avusteinen
1,TI01,"Mahtava moka : uskalla, opi ja menesty",9789521433085,epäonnistuminen,1,kuvailut_tieto Osma,annif_avusteinen
2,TI01,"Mahtava moka : uskalla, opi ja menesty",9789521433085,johtaminen,1,kuvailut_tieto Osma,annif_avusteinen
3,TI01,"Mahtava moka : uskalla, opi ja menesty",9789521433085,pelko,1,kuvailut_tieto Osma,annif_avusteinen
4,TI01,"Mahtava moka : uskalla, opi ja menesty",9789521433085,esimiehet,1,kuvailut_tieto Osma,annif_avusteinen
5,TI01,"Mahtava moka : uskalla, opi ja menesty",9789521433085,organisaatiokulttuuri,1,kuvailut_tieto Osma,annif_avusteinen
6,TI01,"Mahtava moka : uskalla, opi ja menesty",9789521433085,rohkeus,1,kuvailut_tieto Osma,annif_avusteinen
10,TI02,"Väärää vientiä : mene itään tai länteen, mutta...",9789521433849,vienti,1,kuvailut_tieto Osma,annif_avusteinen
11,TI02,"Väärää vientiä : mene itään tai länteen, mutta...",9789521433849,yritykset,1,kuvailut_tieto Osma,annif_avusteinen
12,TI02,"Väärää vientiä : mene itään tai länteen, mutta...",9789521433849,liiketoiminta,1,kuvailut_tieto Osma,annif_avusteinen


In [171]:
df_alaosa0 = pd.DataFrame({'did': [''], 'nimeke': [''], 'url kokotekstiin': [''], 'aiheet': [''], 'kid': ['']})
df_alaosa1 = pd.DataFrame({'did': [''], 'nimeke': [''], 'url kokotekstiin': [''], 'aiheet': ['arvosana'], 'kid': ['']})
df_alaosa2 = pd.DataFrame({'did': ['']*4, 'nimeke': ['']*4, 'url kokotekstiin': ['Kuvailu on kattava', 'Käytetyt asiasanat ovat sisällön kannalta merkityksellisiä', 'Kuvailu on laadukas', 'Vapaa kenttä'], 'aiheet': ['']*4, 'kid': ['']*4})

df_alaosa = pd.concat([df_alaosa0, df_alaosa1, df_alaosa2, df_alaosa0, df_alaosa0])

df_ylaosa = pd.DataFrame({'did': [''], 'kid': [''], 'nimeke': [''], 'url kokotekstiin': [''], 'aiheet': ['aiheet'], 'poistettavat': ['poista aihe merkitsemällä x']})

df_ylaosa.append(df_dokumentti_kuvailu[['did', 'nimeke', 'url kokotekstiin', 'aiheet', 'kid']], sort=False).append(df_alaosa, sort=False).fillna('')

,did,kid,nimeke,url kokotekstiin,aiheet,poistettavat
0,,,,,aiheet,poista aihe merkitsemällä x
0,D00,33,"Vielma, Jouni ym: Sininen biotalous : verkkoju...",http://jukuri.luke.fi/bitstream/handle/10024/5...,biotalous,
1,D00,33,"Vielma, Jouni ym: Sininen biotalous : verkkoju...",http://jukuri.luke.fi/bitstream/handle/10024/5...,luonnonvarat,
2,D00,33,"Vielma, Jouni ym: Sininen biotalous : verkkoju...",http://jukuri.luke.fi/bitstream/handle/10024/5...,uusiutuvat luonnonvarat,
3,D00,33,"Vielma, Jouni ym: Sininen biotalous : verkkoju...",http://jukuri.luke.fi/bitstream/handle/10024/5...,vesistöt,
4,D00,33,"Vielma, Jouni ym: Sininen biotalous : verkkoju...",http://jukuri.luke.fi/bitstream/handle/10024/5...,kestävä kehitys,
5,D00,33,"Vielma, Jouni ym: Sininen biotalous : verkkoju...",http://jukuri.luke.fi/bitstream/handle/10024/5...,vesivarat,
6,D00,33,"Vielma, Jouni ym: Sininen biotalous : verkkoju...",http://jukuri.luke.fi/bitstream/handle/10024/5...,vesiekosysteemit,
0,,,,,,
0,,,,,arvosana,


In [172]:
min_num_aiheet = 1
max_num_ihmiskuvailut = 3
num_arvioija = 2

sisällytä_tyypit = ['fennica', 'annif_avusteinen']

columns = ['did', 'kid', 'nimeke', 'url kokotekstiin', 'aiheet'] #, 'tiedostonimi']

for arvioija_ind in range(num_arvioija):
    spread = Spread('arvioija_' + str(arvioija_ind) , create_spread=True)
    spread.move('Test folder')

    cnt_sheet = 0
    for did, df_dokumentti in df_kuvailut.groupby('did'):
        spread.open_sheet(did, create=True)
        df_pohja = pd.DataFrame(columns=columns)
        print(did)
        kuvailut = [df_dokumentti_kuvailu 
                    for _, df_dokumentti_kuvailu 
                    in df_dokumentti.groupby('kid')
                    if df_dokumentti_kuvailu.tyyppi.iloc[0] in sisällytä_tyypit
                    and len(df_dokumentti_kuvailu.aiheet[df_dokumentti_kuvailu.aiheet != '']) >= min_num_aiheet
                    ]
        ihmiskuvailut = [df_dokumentti_kuvailu 
                    for _, df_dokumentti_kuvailu 
                    in df_dokumentti.groupby('kid')
                    if df_dokumentti_kuvailu.tyyppi.iloc[0] == 'ihminen'
                    and len(df_dokumentti_kuvailu.aiheet[df_dokumentti_kuvailu.aiheet != '']) >= min_num_aiheet
                    ]
        random.shuffle(ihmiskuvailut)
        ihmiskuvailut = ihmiskuvailut[:max_num_ihmiskuvailut]
        kuvailut.extend(ihmiskuvailut)
        random.shuffle(kuvailut)
        
        #df_otsake = pd.DataFrame(kuvailut[0].iloc[:2])
        #spread.df_to_sheet(df_otsake, index=False)
        
        for df_dokumentti_kuvailu in kuvailut:
            #print(df_dokumentti_kuvailu)
            #if len(df_dokumentti_kuvailu.aiheet[df_dokumentti_kuvailu.aiheet != '']) < min_num_aiheet:
            #    continue
            #print(df_dokumentti_kuvailu)
            #df_dokumentti_kuvailu = df_dokumentti_kuvailu[columns].append(df_alaosa, sort=False)
            df_pohja = df_pohja.append(df_ylaosa.append(df_dokumentti_kuvailu[columns], sort=False).append(df_alaosa, sort=False).fillna(''), sort=False)
            #df_pohja = pd.concat([df_pohja, df_ylaosa, df_dokumentti_kuvailu[['did', 'nimeke', 'url kokotekstiin', 'aiheet', 'kid']], df_alaosa])
            
            #print()
        print(df_pohja)
        spread.df_to_sheet(df_pohja, index=False) #, headers=False)
        
        cnt_sheet += 1
        if cnt_sheet > 10: break


D01
     did kid  ...                aiheet                 poistettavat
0             ...                aiheet  poista aihe merkitsemällä x
10   D01  14  ...         näytteleminen                             
11   D01  14  ...       keskittymiskyky                             
12   D01  14  ...           näyttelijät                             
13   D01  14  ...            kokemukset                             
14   D01  14  ...      Suzuki-menetelmä                             
15   D01  14  ...             klovneria                             
0             ...                                                   
0             ...              arvosana                             
0             ...                                                   
1             ...                                                   
2             ...                                                   
3             ...                                                   
0             ...             

0

In [0]:
df_pohja = pd.DataFrame({'col1': [1,2,4], 'col2': [100,200,400] })
spread = Spread('test_arviointi2', create_spread=True)
spread.open_sheet('test sheet', create=True)
spread.df_to_sheet(df_pohja, index=False)

In [0]:
df_dokumentti_kuvailu


,did,nimeke,url kokotekstiin,aiheet,kid
140,D04,Arja Heinonen: Hotellien aamiaispöydissä tarjo...,http://www.hel.fi/static/ymk/julkaisut/julkais...,pedagogiikka,20


In [0]:
[df_dokumentti_kuvailu.tiedostonimi.iloc[0]
                for kid, df_dokumentti_kuvailu 
                in df_dokumentti.groupby('kid')]

['Copy of Kuvailut K31',
 'Copy of Kuvailut Fennica',
 'Copy of Kuvailut K02',
 'Copy of Kuvailut K17',
 'Copy of Kuvailut K05',
 'Copy of Kuvailut K14',
 'Copy of Kuvailut K11',
 'Copy of Kuvailut K08',
 'Copy of Kuvailut K32',
 'Copy of Kuvailut K23',
 'Copy of Kuvailut K29',
 'Copy of Kuvailut K20',
 'Copy of Kuvailut K26',
 'Copy of Kuvailut K19',
 'Copy of Kuvailut K10',
 'Copy of Kuvailut K38',
 'Copy of Kuvailut K13',
 'Copy of Kuvailut K35',
 'Copy of Kuvailut K40',
 'Copy of Kuvailut K28',
 'Copy of Kuvailut K25',
 'Copy of Kuvailut K37',
 'Copy of Kuvailut K22',
 'Copy of Kuvailut K18',
 'Copy of Kuvailut K06',
 'Copy of Kuvailut K12',
 'Copy of Kuvailut K03',
 'Copy of Kuvailut K09',
 'Copy of Kuvailut K33',
 'Copy of Kuvailut K39',
 'Copy of Kuvailut K21',
 'Copy of Kuvailut K36',
 'Copy of Kuvailut K04',
 'Copy of Kuvailut K01',
 'Copy of Kuvailut K34']

In [0]:
len(kuvailut)

12

In [0]:
# create a blank spreadsheet with a specified title.

spreadsheet = {
    'properties': {
        'title': title
    }
}
spreadsheet = service.spreadsheets().create(body=spreadsheet,
                                    fields='spreadsheetId').execute()
print('Spreadsheet ID: {0}'.format(spreadsheet.get('spreadsheetId')))


#To write to a sheet, you will need the spreadsheet ID, the range(s) in A1 notation,
#and the data you wish to write arranged in an appropriate request body object.
values = [
    [
        # Cell values ...
    ],
    # Additional rows
]
data = [
    {
        'range': range_name,
        'values': values
    },
    # Additional ranges to update ...
]
body = {
    'valueInputOption': value_input_option,
    'data': data
}
result = service.spreadsheets().values().batchUpdate(
    spreadsheetId=spreadsheet_id, body=body).execute()
print('{0} cells updated.'.format(result.get('totalUpdatedCells')))


#copy sheets
from googleapiclient import discovery

# TODO: Change placeholder below to generate authentication credentials. See
# https://developers.google.com/sheets/quickstart/python#step_3_set_up_the_sample
#
# Authorize using one of the following scopes:
#     'https://www.googleapis.com/auth/drive'
#     'https://www.googleapis.com/auth/drive.file'
#     'https://www.googleapis.com/auth/spreadsheets'
credentials = None

service = discovery.build('sheets', 'v4', credentials=credentials)

# The ID of the spreadsheet containing the sheet to copy.
spreadsheet_id = 'my-spreadsheet-id'  # TODO: Update placeholder value.

# The ID of the sheet to copy.
sheet_id = 0  # TODO: Update placeholder value.

copy_sheet_to_another_spreadsheet_request_body = {
    # The ID of the spreadsheet to copy the sheet to.
    'destination_spreadsheet_id': '',  # TODO: Update placeholder value.

    # TODO: Add desired entries to the request body.
}

request = service.spreadsheets().sheets().copyTo(spreadsheetId=spreadsheet_id, sheetId=sheet_id, body=copy_sheet_to_another_spreadsheet_request_body)
response = request.execute()

# TODO: Change code below to process the `response` dict:


In [0]:
#using gspread
scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive'
]

# making spreadsheets
create(title)

t == ""
for i in range(50):
  t == t+str(i)
  create()

